In [17]:
import pandas as pd
import numpy as np
from copy import copy
import datetime
import pickle
from geopy.geocoders import Nominatim

In [26]:
filename = "LightGBM_model.sav"

In [27]:
loaded_model = pickle.load(open(filename, 'rb'))

In [18]:
#Sample date

date_list = [4, 6, 2016] 
#April 6, 2016

year = int(date_list[2])
month = int(date_list[1])
day = int(date_list[0])

my_date = datetime.date(year, month, day)

In [19]:
#Sample test locations

test_locations = {'L1': (40.819688, -73.915091),
                  'L2': (40.815421, -73.941761),
                  'L3': (40.764198, -73.910785),
                  'L4': (40.768790, -73.953285),
                  'L5': (40.734851, -73.952950),
                  'L6': (40.743613, -73.977998),
                  'L7': (40.745313, -73.993793),
                  'L8': (40.662713, -73.946101),
                  'L9': (40.703761, -73.886496),
                  'L10': (40.713620, -73.943076),
                  'L11': (40.725212, -73.809179)
             }

In [20]:
geolocator = Nominatim()
addresses = []

for key in test_locations:
    location = geolocator.reverse(test_locations[key])
    addresses.append(location.address)

/usr/local/lib/python3.7/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


In [21]:
addresses

['424, East 155th Street, Melrose, The Bronx, Bronx County, New York, 10455, United States',
 '137, West 136th Street, Harlem, Manhattan, New York County, New York, 10030, United States',
 'Citi Bike - 28 Av / 44 St, 28th Avenue, Queens, Queens County, New York, 11377, United States',
 '435, East 74th Street, Lenox Hill, Manhattan, New York County, New York, 10021, United States',
 '211, Freeman Street, Brooklyn, Kings County, New York, 11222, United States',
 '232, East 32nd Street, Kips Bay, Manhattan Community Board 6, Manhattan, New York County, New York, 10016, United States',
 '159, West 25th Street, Chelsea, Manhattan, New York County, New York, 10001, United States',
 '458, Lefferts Avenue, Crown Heights, Brooklyn, Kings County, New York, 11225, United States',
 '70-38, 67th Place, Glendale, Queens, Queens County, New York, 11385, United States',
 '194, Devoe Street, Williamsburg, Brooklyn, Kings County, New York, 11211, United States',
 '158-46, 76th Avenue, Utopia, Queens, Qu

In [22]:
test_addresses = {'L1': '424 East 155th Street NY',
                  'L2': '137 West 136th Street NY',
                  'L3': '43-11 28th Avenue NY',
                  'L4': '435 East 74th Street NY',
                  'L5': '211 Freeman Street NY',
                  'L6': '232 East 32nd Street NY',
                  'L7': '159 West 25th Street NY',
                  'L8': '486 Brooklyn Avenue NY',
                  'L9': '70-38 67th Place NY',
                  'L10': '194 Devoe Street NY',
                  'L11': '158-46 76th Avenue NY'
             }

####  List of Point IDs
Creating a random path between all points and then return to the original.

In [23]:
def create_guess(points):
    guess = copy(points)
    np.random.shuffle(guess)
    guess.append(guess[0])
    return list(guess)

create_guess(list(test_locations.keys()))

['L1', 'L6', 'L3', 'L8', 'L4', 'L10', 'L9', 'L5', 'L7', 'L2', 'L11', 'L1']

####  List of lists of all routes.
Makes a list of guessed point orders given a list of point IDs.

In [24]:
def create_generation(points, population=100):
    generation = [create_guess(points) for _ in range(population)]
    return generation

test_generation = create_generation(list(test_locations.keys()), population=10)
print(test_generation)

[['L1', 'L7', 'L5', 'L11', 'L9', 'L6', 'L2', 'L3', 'L4', 'L10', 'L8', 'L1'], ['L9', 'L2', 'L3', 'L6', 'L5', 'L4', 'L10', 'L11', 'L1', 'L8', 'L7', 'L9'], ['L10', 'L1', 'L4', 'L2', 'L5', 'L9', 'L8', 'L7', 'L3', 'L11', 'L6', 'L10'], ['L7', 'L9', 'L2', 'L6', 'L4', 'L1', 'L3', 'L10', 'L5', 'L11', 'L8', 'L7'], ['L5', 'L2', 'L4', 'L10', 'L11', 'L7', 'L6', 'L9', 'L3', 'L1', 'L8', 'L5'], ['L1', 'L2', 'L10', 'L4', 'L3', 'L9', 'L5', 'L8', 'L11', 'L6', 'L7', 'L1'], ['L5', 'L7', 'L8', 'L9', 'L1', 'L10', 'L6', 'L3', 'L4', 'L11', 'L2', 'L5'], ['L11', 'L10', 'L5', 'L8', 'L6', 'L3', 'L1', 'L7', 'L4', 'L9', 'L2', 'L11'], ['L5', 'L2', 'L10', 'L3', 'L1', 'L11', 'L7', 'L8', 'L6', 'L9', 'L4', 'L5'], ['L11', 'L5', 'L2', 'L9', 'L3', 'L8', 'L1', 'L10', 'L7', 'L6', 'L4', 'L11']]


#### Calculate Distance
Given two points, this calculates travel between them based on a LightGBM predictive model

In [25]:
!pip install geopy
from geopy.distance import geodesic
def get_distance(source_lat, source_long, dest_lat, dest_long):
    #distance in Miles between the source and the destination.
    
    distance = geodesic((source_lat, source_long),(dest_lat, dest_long)).miles
    return distance

In [31]:
def travel_time_between_points(point1_id, point2_id, hour, date, passenger_count = 1, 
                               store_and_fwd_flag = 0, pickup_minute = 0):

    
    model_data = {'passenger_count': passenger_count,
                  'pickup_longitude' : point1_id[1],
                  'pickup_latitude' : point1_id[0],
                  'dropoff_longitude' : point2_id[1],
                  'dropoff_latitude' : point2_id[0],
                  'store_and_fwd_flag' : store_and_fwd_flag,
                  'pickup_month' : my_date.month,
                  'pickup_day' : my_date.day,
                  'pickup_weekday' : my_date.weekday(),
                  'pickup_hour': hour,
                  'pickup_minute' : pickup_minute,
                  'latitude_difference' : point2_id[0] - point1_id[0],
                  'longitude_difference' : point2_id[1] - point1_id[1],
                  'trip_distance' : get_distance(point1_id[0], point1_id[1], point2_id[0], point2_id[1])
                 }

    df = pd.DataFrame([model_data], columns=model_data.keys())
  
    pred = np.exp(loaded_model.predict(df)) - 1
    
    return pred[0]

In [32]:
coordinates = test_locations

Find the best Loop

In [33]:
def fitness_score(guess):
    score = 0
    for ix, point_id in enumerate(guess[:-1]):
        score += travel_time_between_points(coordinates[point_id], coordinates[guess[ix+1]], 11, my_date)
    return score

def check_fitness(guesses):
    fitness_indicator = []
    for guess in guesses:
        fitness_indicator.append((guess, fitness_score(guess)))
    return fitness_indicator

print(check_fitness(test_generation))

[(['L1', 'L7', 'L5', 'L11', 'L9', 'L6', 'L2', 'L3', 'L4', 'L10', 'L8', 'L1'], 284.5996831378794), (['L9', 'L2', 'L3', 'L6', 'L5', 'L4', 'L10', 'L11', 'L1', 'L8', 'L7', 'L9'], 323.35937428432413), (['L10', 'L1', 'L4', 'L2', 'L5', 'L9', 'L8', 'L7', 'L3', 'L11', 'L6', 'L10'], 303.14130773910875), (['L7', 'L9', 'L2', 'L6', 'L4', 'L1', 'L3', 'L10', 'L5', 'L11', 'L8', 'L7'], 289.21570384022294), (['L5', 'L2', 'L4', 'L10', 'L11', 'L7', 'L6', 'L9', 'L3', 'L1', 'L8', 'L5'], 300.35247961173366), (['L1', 'L2', 'L10', 'L4', 'L3', 'L9', 'L5', 'L8', 'L11', 'L6', 'L7', 'L1'], 286.5823489772623), (['L5', 'L7', 'L8', 'L9', 'L1', 'L10', 'L6', 'L3', 'L4', 'L11', 'L2', 'L5'], 313.05937390012906), (['L11', 'L10', 'L5', 'L8', 'L6', 'L3', 'L1', 'L7', 'L4', 'L9', 'L2', 'L11'], 302.16587647828084), (['L5', 'L2', 'L10', 'L3', 'L1', 'L11', 'L7', 'L8', 'L6', 'L9', 'L4', 'L5'], 331.41744476261374), (['L11', 'L5', 'L2', 'L9', 'L3', 'L8', 'L1', 'L10', 'L7', 'L6', 'L4', 'L11'], 317.473286329136)]
